# 원스토어 IAP 기술문서 최적화 RAG 시스템
# OneStore IAP Technical Documentation Optimized RAG System

이 노트북은 원스토어 인앱결제 기술문서를 위한 최적화된 RAG 시스템을 데모합니다.

## 🎯 주요 특징
- **계층적 문서 분할**: 출처별 → 헤더별 → 청크별 구조화
- **컨텍스트 인식 검색**: 전문용어의 맥락적 의미 고려
- **다단계 리랭킹**: 관련성, 맥락, 기술정확성 종합 평가
- **Ollama 최적화**: bge-m3 + exaone3.5 로컬 실행


In [24]:
# 환경 설정 및 모듈 import
import os
import sys
import time
import json
from pathlib import Path

# 현재 디렉토리를 Python 경로에 추가
current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.append(str(current_dir))

print(f"작업 디렉토리: {current_dir}")
print(f"Python 경로에 추가됨: {str(current_dir)}")

# 모듈 import
try:
    from optimized_rag_pipeline import OptimizedRAGPipeline, create_pipeline
    from hierarchical_splitter import HierarchicalSplitter
    from context_aware_retriever import ContextAwareRetriever
    print("✅ 모든 모듈 import 성공!")
except ImportError as e:
    print(f"❌ 모듈 import 실패: {e}")
    print("\n필요한 의존성을 설치해주세요:")
    print("pip install -r requirements.txt")
    print("\nOllama 모델도 설치해주세요:")
    print("ollama pull bge-m3:latest")
    print("ollama pull exaone3.5:latest")


작업 디렉토리: /home/gpu/dev/jupyter_root
Python 경로에 추가됨: /home/gpu/dev/jupyter_root
❌ 모듈 import 실패: No module named 'optimized_rag_pipeline'

필요한 의존성을 설치해주세요:
pip install -r requirements.txt

Ollama 모델도 설치해주세요:
ollama pull bge-m3:latest
ollama pull exaone3.5:latest


In [ ]:
# 파이프라인 초기화 및 테스트
data_file = "../../data/dev_center_guide_allmd_touched.md"

# 데이터 파일 확인
if os.path.exists(data_file):
    print(f"✅ 데이터 파일 확인됨: {data_file}")
    
    # 파이프라인 생성
    print("🔧 파이프라인 초기화 중...")
    pipeline = create_pipeline(data_file, force_rebuild=False)
    
    # 테스트 질문
    test_question = "PurchaseClient를 어떻게 초기화하나요?"
    print(f"\n🤔 테스트 질문: {test_question}")
    
    result = pipeline.query(test_question, stream=False)
    
    print(f"\n💡 답변:")
    print(result['answer'])
    
    print(f"\n📊 통계:")
    stats = pipeline.get_statistics()
    print(f"  - 총 문서 청크: {stats['total_chunks']}")
    print(f"  - 응답 시간: {result['query_time']:.2f}초")
    
else:
    print(f"❌ 데이터 파일을 찾을 수 없습니다: {data_file}")
    print("경로를 확인하거나 파일을 복사해주세요.")


In [ ]:
# 사용자 질문 테스트 (pipeline이 초기화된 후 실행)
if 'pipeline' in locals():
    print("💬 질문을 입력하여 테스트해보세요!")
    
    # 추가 테스트 질문들
    test_questions = [
        "purchaseState 값은 무엇인가요?",
        "PNS 서비스란 무엇인가요?",
        "구독형 상품과 관리형 상품의 차이점은?",
        "인앱결제 테스트는 어떻게 하나요?"
    ]
    
    print("\n📝 추천 질문들:")
    for i, q in enumerate(test_questions, 1):
        print(f"  {i}. {q}")
    
    print("\n아래에 원하는 질문을 입력하고 실행하세요:")
    print("user_question = '질문을 여기에 입력'")
    print("result = pipeline.query(user_question, stream=False)")
    print("print(result['answer'])")
    
else:
    print("❌ 먼저 위의 셀을 실행하여 파이프라인을 초기화하세요.")


In [ ]:
# 여기에 원하는 질문을 입력하고 테스트하세요
user_question = "purchaseState 값은 무엇인가요?"

if 'pipeline' in locals():
    result = pipeline.query(user_question, stream=False)
    
    print(f"🤔 질문: {user_question}")
    print("=" * 60)
    print(f"💡 답변:")
    print(result['answer'])
    print(f"\n⏱️ 응답 시간: {result['query_time']:.2f}초")
    print(f"📄 참조 문서: {result['retrieved_docs_count']}개")
else:
    print("❌ 파이프라인이 초기화되지 않았습니다. 위의 셀들을 먼저 실행하세요.")
